### Лабораторная работа 4 
### Тема: Детектирование движения и отслеживание на видео 

1. Разработать программу детектирования движения на видео методом 
вычитания модели фона (Background Subtraction Methods).  
При возникновении движения в поле зрения камеры (или на 
видео) подается сигнал (или выводится сообщение).  
Пример из жизни – звуковая сигнализация при обнаружении движения, или 
владельцу наблюдаемого помещения приходит смс с сообщением.

Что такое детекция движения и зачем она нужна
Детекция движения — это процесс в компьютерном зрении, когда программа анализирует видео или поток с камеры, чтобы понять, есть ли в кадре что-то движущееся. Это как "глаза" для компьютера, которые помогают замечать изменения. Полезно в системах безопасности (например, камеры видеонаблюдения, которые включают тревогу при движении), робототехнике (робот реагирует на приближающихся людей), трафик-контроле (считает машины) или даже в играх и приложениях для фитнеса (отслеживает движения тела). В твоём коде это реализовано просто: программа смотрит на видео, выделяет фон (то, что не меняется), и если что-то меняется — сигнализирует о движении.
Теория основана на идее, что в статичной сцене (например, пустая комната) фон остаётся примерно одинаковым, а движение — это отклонения от этого фона. Главный метод здесь — вычитание фона (background subtraction). Это когда из текущего кадра "вычитают" модель фона, и то, что остаётся, считается передним планом (foreground), то есть движущимися объектами.
Ключевые термины (терминология)
Давай разберём простыми словами основные понятия, которые встречаются в коде и теории:

Фон (Background): Это статичная часть сцены, которая не меняется со временем. Например, стены, пол, мебель в комнате. Программа учится понимать, что это "нормально" и неинтересно.
Передний план (Foreground): Движущиеся объекты, которые отличаются от фона. Например, человек, проходящий мимо, или машина на дороге.
Маска переднего плана (Foreground Mask или FG Mask): Это чёрно-белое изображение, где белые пиксели — это движение (передний план), а чёрные — фон. В коде это fgmask — как "карта" изменений.
Вычитание фона (Background Subtraction): Основной алгоритм. Он строит модель фона на основе предыдущих кадров и сравнивает новый кадр с этой моделью. Если пиксель в новом кадре сильно отличается — он помечается как движение.
MOG2 (Mixture of Gaussians 2): Это конкретный алгоритм вычитания фона, который используется в коде (cv2.createBackgroundSubtractorMOG2()). "Mixture of Gaussians" значит "смесь гауссовых распределений" — математическая модель, которая описывает, как могут варьироваться цвета пикселей на фоне (учитывает шум, освещение и т.д.). "2" — это улучшенная версия, которая лучше справляется с тенями и изменениями света.
Порог (Threshold): Число, которое определяет, когда считать изменение "движением". В коде это motion > 1000 — если белых пикселей в маске больше 1000, то движение обнаружено. Это чтобы игнорировать мелкий шум, как дрожание камеры или лёгкие блики.
Кадр (Frame): Один снимок из видео. Видео — это последовательность кадров, как слайд-шоу.
Пиксель (Pixel): Маленькая точка на изображении, из которых состоит весь кадр. Каждый имеет цвет (RGB — красный, зелёный, синий).
Шум (Noise): Нежелательные помехи, как случайные изменения в кадре от плохого освещения или сенсора камеры. Алгоритмы вроде MOG2 пытаются их игнорировать.
OpenCV: Библиотека (набор готовых функций) для компьютерного зрения. В коде это cv2 — она делает всю тяжёлую работу, как чтение видео, применение алгоритмов и показ изображений.

Как работает метод в коде (пошагово и просто)
Твой код использует MOG2 для вычитания фона. Вот как это работает на теории и в практике:


Инициализация: Программа открывает видео или камеру (cap = cv2.VideoCapture(0) — 0 значит веб-камера). Создаёт объект MOG2 (fgbg = cv2.createBackgroundSubtractorMOG2()). Этот объект — как "мозг", который будет учиться на кадрах.


Обработка каждого кадра: В цикле while программа читает кадр за кадром (ret, frame = cap.read()). Если кадр есть, применяет MOG2: fgmask = fgbg.apply(frame).

Что делает MOG2 внутри:

Для каждого пикселя в кадре MOG2 смотрит на историю предыдущих кадров. Он моделирует фон как смесь нескольких "гауссов" — это как нормальные распределения в статистике, где цвета пикселей могут слегка варьироваться (например, из-за света).
Если пиксель в новом кадре вписывается в модель фона (похож на то, что было раньше) — он чёрный в маске (фон).
Если не вписывается (сильно отличается) — белый (передний план, движение).
MOG2 адаптируется: если что-то статично долго (например, стул передвинули и оставили), оно со временем станет частью фона.
Плюс, MOG2 обнаруживает тени и игнорирует их, чтобы не путать с реальным движением.





Проверка на движение: Считает белые пиксели в маске (motion = cv2.countNonZero(fgmask)). Если их много (больше 1000) — значит, движение! Добавляет текст на кадр (cv2.putText) и печатает "Сигнал: движение!".


Визуализация: Показывает оригинальный кадр (cv2.imshow('Frame')) и маску (cv2.imshow('FG Mask')). Ждёт нажатия Esc для выхода.


Завершение: Закрывает видео и окна.


В целом, это работает как "детектор изменений": фон — стабильный, движение — отклонение. Но это не идеально: если фон меняется (например, день/ночь), или объекты двигаются медленно — могут быть ошибки. Порог 1000 — это тюнинг, чтобы избежать ложных срабатываний от шума.
Что ещё есть: альтернативы и расширения
Кроме MOG2, есть другие способы детекции движения. Я расскажу не слишком глубоко, но подробно, простыми словами:

Другие алгоритмы вычитания фона в OpenCV:

MOG (первый вариант): Похож на MOG2, но хуже справляется с тенями и изменениями освещения. MOG2 — улучшенная версия.
KNN (K-Nearest Neighbors): Ещё один метод в OpenCV (cv2.createBackgroundSubtractorKNN()). Работает на основе "ближайших соседей" — сравнивает пиксели с похожими из истории. Лучше для сцен с медленными изменениями, но может быть медленнее MOG2. Можно заменить в коде и сравнить.
GMG (Godbehere-Matsukawa-Goldberg): Комбинирует статистики и байесовские модели. Хорош для шумных видео, но требует больше вычислений.


Альтернативные подходы без вычитания фона:

Оптический поток (Optical Flow): Смотрит, как пиксели "текут" между кадрами (как вода). Методы вроде Lucas-Kanade или Farneback в OpenCV. Полезно для отслеживания направления движения, но сложнее и медленнее. Не выделяет фон, а фокусируется на векторах движения.
Разница кадров (Frame Differencing): Простой способ — вычитаешь предыдущий кадр из текущего. Дёшево, но не справляется с медленным движением или если камера двигается. В коде можно добавить diff = cv2.absdiff(prev_frame, frame) и thresholding.
Контурный анализ: После маски (как в твоём коде) можно найти контуры (cv2.findContours) — границы объектов. Затем фильтровать по размеру, чтобы игнорировать мелкие движения (например, листья на ветру).
Машинное обучение: Современные методы, как YOLO или Mask R-CNN, используют нейросети для детекции объектов (не просто движения, а "это человек" или "машина"). Они точнее, но требуют GPU и обучения. В OpenCV есть интеграция с TensorFlow или PyTorch.


Улучшения для твоего кода:

Фильтрация шума: После маски применить эрозию/дилятацию (cv2.erode, cv2.dilate) — чтобы убрать мелкие белые точки.
Отслеживание объектов: Использовать Kalman Filter или SORT, чтобы следить за одним и тем же объектом через кадры (не просто "движение", а "человек идёт вправо").
Адаптация к изменениям: MOG2 имеет параметры, как history (сколько кадров помнить) или detectShadows (включить/выключить тени). По умолчанию тени детектируются как серые в маске.
Проблемы и решения: Если освещение меняется — используй адаптивные пороги. Для движущейся камеры (дрон) — нужны более сложные методы, как SLAM.



В общем, MOG2 — хороший старт для простых задач, но для реальных приложений комбинируют несколько методов. Если хочешь, могу помочь доработать код или объяснить что-то конкретнее!

In [ ]:
%pip uninstall opencv-python opencv-contrib-python -y
%pip install opencv-python numpy
import cv2

# Видео можно заменить на 0 для веб-камеры
cap = cv2.VideoCapture(0)  # Замените на путь к своему видео или 0 для веб-камеры

# Создание объекта для вычитания фона (MOG2 наиболее часто используется)
fgbg = cv2.createBackgroundSubtractorMOG2()
iter = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    fgmask = fgbg.apply(frame)

    # Определение, есть ли движение (по наличию белых пикселей)
    motion = cv2.countNonZero(fgmask)
    if motion > 1000:  # Порог, чтобы не реагировать на шум

        cv2.putText(frame, f"Detected motion: {iter}", (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
        iter += 1 
        print('Сигнал: движение!')

    cv2.imshow('Frame', frame)
    cv2.imshow('FG Mask', fgmask)
    if cv2.waitKey(30) & 0xFF == 27:  # Esc для выхода
        break

cap.release()
cv2.destroyAllWindows()

Что улучшено в коде

Проверка открытия камеры: Добавлена проверка cap.isOpened() в начале, чтобы сразу сообщить об ошибке, если камера недоступна.
Настройка MOG2:

history=500: Увеличено количество кадров для модели фона, чтобы лучше адаптироваться к изменениям.
varThreshold=16: Порог для классификации пикселей (фон/передний план) оставлен стандартным, но можно настроить для чувствительности.
detectShadows=True: Тени отображаются серым, что помогает отличать их от реальных объектов.


Фильтрация шума:

Добавлены морфологические операции (cv2.morphologyEx):

MORPH_OPEN: Удаляет мелкие белые точки (шум) в маске.
MORPH_CLOSE: Заполняет мелкие разрывы в движущихся объектах.


Используется эллиптическое ядро (cv2.getStructuringElement) размером 5x5 для сглаживания.


Контурный анализ:

Вместо подсчёта белых пикселей (cv2.countNonZero) добавлен поиск контуров (cv2.findContours).
Контуры с площадью меньше min_contour_area (500 пикселей) игнорируются, чтобы исключить шум.
Для каждого значимого контура рисуется зелёный прямоугольник (cv2.boundingRect), что визуально показывает, где именно обнаружено движение.


Улучшенная логика детекции:

Движение считается обнаруженным, если есть контуры с достаточной площадью и общее количество белых пикселей превышает min_motion_area (1000).
Это делает детекцию более надёжной, исключая ложные срабатывания от мелких шумов.


Информативный вывод:

В сообщение о движении добавлен номер итерации (motion_counter) для ясности.
Проверка на ошибку чтения кадра с выводом сообщения.



Зачем эти улучшения

Снижение шума: Морфологические операции и контурный анализ делают детекцию точнее, исключая мелкие помехи (например, дрожание света).
Визуализация объектов: Прямоугольники вокруг движущихся объектов показывают, где конкретно происходит движение, что полезно для отладки.
Гибкость MOG2: Настроенные параметры позволяют лучше адаптироваться к разным условиям (например, с тенями).
Надёжность: Проверки на ошибки делают код устойчивым к сбоям камеры или видео.

Как тестировать и настраивать

Порог min_contour_area: Увеличьте (например, до 1000), если детектируются мелкие объекты (листья, блики). Уменьшите (до 200), если пропускаются маленькие объекты.
Порог min_motion_area: Похож на оригинальный порог 1000. Уменьшите для большей чувствительности, увеличьте для меньшей.
Ядро морфологии: Размер kernel (5,5) можно увеличить до (7,7) для более агрессивного подавления шума, но это может убрать мелкие объекты.
Параметры MOG2:

Увеличьте history (например, до 1000), если фон меняется медленно.
Уменьшите varThreshold (до 10), чтобы сделать детекцию чувствительнее, или увеличьте (до 25) для меньшей чувствительности.

In [ ]:
import cv2
import numpy as np

# Открытие видеопотока (0 для веб-камеры)
cap = cv2.VideoCapture(0)

# Проверка успешного открытия камеры
if not cap.isOpened():
    print("Ошибка: Не удалось открыть камеру.")
    exit()

# Создание объекта MOG2 с настройками
fgbg = cv2.createBackgroundSubtractorMOG2(
    history=500,           # Количество кадров для модели фона
    varThreshold=16,       # Порог для классификации пикселей
    detectShadows=True     # Обнаружение теней
)

# Параметры морфологических операций для фильтрации шума
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
motion_counter = 0
min_motion_area = 1000  # Порог площади движения
min_contour_area = 500  # Минимальная площадь контура для исключения шума

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Ошибка: Не удалось захватить кадр.")
        break

    # Применение вычитания фона
    fgmask = fgbg.apply(frame)

    # Фильтрация шума с помощью морфологических операций
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)  # Удаление мелкого шума
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_CLOSE, kernel) # Заполнение мелких разрывов

    # Подсчёт ненулевых пикселей
    motion = cv2.countNonZero(fgmask)

    # Поиск контуров для более точной детекции
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    motion_detected = False

    # Анализ контуров
    for contour in contours:
        if cv2.contourArea(contour) > min_contour_area:
            motion_detected = True
            # Отрисовка прямоугольника вокруг движущегося объекта
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Если обнаружено движение
    if motion_detected and motion > min_motion_area:
        cv2.putText(
            frame, f"Detected motion: {motion_counter}", (10, 40),
            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2
        )
        motion_counter += 1
        print(f"Сигнал: движение! (Итерация: {motion_counter})")

    # Показ кадра и маски
    cv2.imshow('Frame', frame)
    cv2.imshow('FG Mask', fgmask)

    # Выход по Esc
    if cv2.waitKey(30) & 0xFF == 27:
        break

# Освобождение ресурсов
cap.release()
cv2.destroyAllWindows()

2. Метод Лукаса-Канаде. Визуализировать разреженный оптический 
поток с помощью функции calcOpticalFlowPyrLK () с отрисовкой 
траектории движения. 

Что такое оптический поток и зачем он нужен
Оптический поток (Optical Flow) — это метод в компьютерном зрении, который анализирует, как движутся объекты (или их части) между кадрами видео. Он не просто говорит "есть движение", а показывает, куда и как быстро движутся отдельные точки или объекты. Это как если бы ты смотрел на видео и видел стрелки, показывающие, в какую сторону движется каждый пиксель. Применяется в:

Робототехнике: для навигации роботов или дронов (например, чтобы понять, куда движется препятствие).
Слежение за объектами: в камерах видеонаблюдения или в спортивных приложениях (отслеживать игроков).
Стабилизация видео: чтобы убрать дрожание камеры.
Анализ движения: например, в играх или медицинских системах для анализа походки.

Твой код использует метод Лукаса-Канаде (Lucas-Kanade) для вычисления оптического потока, отслеживая движение ключевых точек в кадрах. Он рисует траектории движения точек зелёными линиями и отмечает текущие позиции точек красными кругами. Давай разберём теорию, термины и как это работает простыми словами.

Ключевые термины (терминология)

Оптический поток (Optical Flow): Векторное поле, показывающее, как каждый пиксель (или группа пикселей) перемещается между кадрами. Это как "карта движения", где для каждой точки есть направление и скорость.
Ключевые точки (Keypoints): Особые точки в кадре, которые легко отслеживать (например, углы, края объектов). В коде они находятся с помощью cv2.goodFeaturesToTrack.
Метод Лукаса-Канаде (Lucas-Kanade): Алгоритм для вычисления оптического потока. Он предполагает, что движение маленькое и что яркость пикселей не меняется между кадрами. Отслеживает движение только для выбранных точек, а не всего кадра.
goodFeaturesToTrack: Функция OpenCV, которая ищет "хорошие" точки для отслеживания (обычно углы, где много контраста). Это как выбрать заметные ориентиры на изображении.
Пирамида изображений (Image Pyramid): Метод, где изображение обрабатывается в нескольких масштабах (от мелкого к крупному), чтобы лучше находить движение, даже если оно большое. В коде это задаётся через maxLevel в параметрах Лукаса-Канаде.
Маска (Mask): В коде — это изображение того же размера, что и кадр, на котором рисуются траектории (зелёные линии). Оно "накладывается" на оригинальный кадр, чтобы показать движение.
Пиксель (Pixel): Маленькая точка изображения с цветом (RGB) или яркостью (в градациях серого).
WinSize (размер окна): Область вокруг ключевой точки, которую алгоритм анализирует для поиска движения. В коде это (15,15) — квадрат 15x15 пикселей.
Критерии остановки (Criteria): Условия, когда алгоритм Лукаса-Канаде прекращает искать движение точки. В коде это комбинация точности (cv2.TERM_CRITERIA_EPS) и числа итераций (cv2.TERM_CRITERIA_COUNT).
OpenCV: Библиотека для компьютерного зрения, которая делает всю работу — от чтения кадров до вычисления потока.


Как работает метод в коде (пошагово и просто)
Твой код реализует метод Лукаса-Канаде для отслеживания движения ключевых точек. Вот что происходит:

Инициализация:

Программа открывает веб-камеру (cv2.VideoCapture(0)).
Берёт первый кадр (old_frame) и переводит его в градации серого (old_gray), так как Лукасу-Канаде не нужны цвета, только яркость.
Находит ключевые точки с помощью cv2.goodFeaturesToTrack (до 100 углов, с параметрами качества, расстояния и размера блока). Это точки, которые легко отслеживать (например, углы объектов).
Создаёт пустую маску (np.zeros_like(old_frame)) для рисования траекторий.


Цикл обработки кадров:

Читает новый кадр (frame) и переводит его в градации серого (frame_gray).
Использует cv2.calcOpticalFlowPyrLK для вычисления оптического потока:

Сравнивает old_gray (предыдущий кадр) и frame_gray (текущий).
Для каждой ключевой точки (p0) ищет, куда она сместилась (p1).
Возвращает новые координаты точек (p1), статус (st, 1 — точка найдена, 0 — потеряна) и ошибку (err).


Если точки найдены (p1 и st не пустые):

Фильтрует точки, где st == 1 (успешно отслеженные).
Для каждой пары старой и новой точки рисует:

Зелёную линию на маске (cv2.line) от старой позиции (good_old) к новой (good_new).
Красный круг на текущем кадре (cv2.circle) в новой позиции.


Складывает кадр с маской (cv2.add), чтобы показать и видео, и траектории.


Показывает результат в окне (cv2.imshow).
Обновляет предыдущий кадр (old_gray = frame_gray) и ищет новые ключевые точки (p0) для следующей итерации.


Выход: Цикл прерывается при нажатии Esc или если видео кончилось.

Как работает Лукас-Канаде внутри:

Предполагает, что яркость пикселей не меняется между кадрами (если точка была яркой, она останется такой же).
Смотрит на маленькое окно вокруг каждой ключевой точки (15x15 пикселей) и ищет, где это окно "похоже" в новом кадре.
Использует пирамиду изображений (maxLevel=2), чтобы сначала искать движение на грубом уровне (низкое разрешение), а потом уточнять.
Если точка потерялась (например, вышла за кадр), она исключается (st == 0).

Почему точки выбираются заново:
В коде p0 обновляется на каждом кадре (cv2.goodFeaturesToTrack). Это нужно, потому что точки могут "теряться" (выходят за кадр, закрываются объектом). Обновление помогает находить новые интересные точки.

Теория: как Лукас-Канаде вычисляет движение
Метод Лукаса-Канаде основан на двух предположениях:

Постоянство яркости: Цвет или яркость точки не меняется при движении. Если угол стола был серым, он останется серым, даже если сместится.
Малое движение: Точки двигаются не слишком далеко между кадрами (например, в пределах окна 15x15 пикселей).

Алгоритм:

Берёт окно вокруг ключевой точки в старом кадре.
Ищет, где это окно "лучше всего" совпадает в новом кадре, минимизируя разницу в яркости.
Использует градиенты изображения (как меняется яркость по x и y), чтобы вычислить направление и величину смещения.
Пирамида помогает: сначала смотрит на маленькое изображение (где большие движения выглядят маленькими), потом уточняет на полном.

Ограничения:

Не работает, если движение слишком быстрое (точка уходит за пределы окна).
Плохо справляется с большими изменениями освещения.
Требует "заметных" точек (углов, краёв), иначе нечего отслеживать.


Что ещё есть: альтернативы и расширения
Есть другие подходы к анализу движения, которые могут дополнить или заменить Лукаса-Канаде:

Другие методы оптического потока:

Farneback: Метод в OpenCV (cv2.calcOpticalFlowFarneback). Вычисляет поток для всех пикселей, а не только ключевых точек. Даёт плотное поле (вектор для каждого пикселя), но медленнее. Подходит для анализа общего движения сцены.
DeepFlow или FlowNet: Нейросетевые методы, которые точнее, но требуют мощного оборудования и обучения. Не в OpenCV, но есть в PyTorch/TensorFlow.


Альтернативы без оптического потока:

Вычитание фона (как в твоём первом коде с MOG2): Просто говорит, есть движение или нет, но не даёт направления.
Контурный анализ: После потока можно найти контуры движущихся объектов (cv2.findContours) и отслеживать их.
Трекинг объектов: Алгоритмы вроде KCF или CSRT в OpenCV (cv2.TrackerKCF_create) отслеживают конкретный объект, а не точки.


Улучшения для твоего кода:

Фильтрация точек: Использовать err (ошибку) из calcOpticalFlowPyrLK, чтобы отбрасывать точки с большой ошибкой.
Сохранение траекторий: Сейчас маска "накапливает" все линии. Можно очищать её через N кадров, чтобы видеть только недавние движения.
Анализ направления: Из векторов (good_new - good_old) можно вычислить, куда движется объект (например, "вправо", "вверх").
Цветовая сегментация: Добавить фильтр по цвету, чтобы отслеживать только определённые объекты (например, красный мяч).
Проблемы и решения: Если освещение меняется, можно нормализовать яркость кадров. Если движение слишком быстрое, увеличить winSize или maxLevel.


Параметры для тюнинга:

maxCorners: Больше точек — больше данных, но медленнее.
qualityLevel: Чем выше, тем меньше, но качественнее точки.
winSize: Большее окно лучше для быстрых движений, но теряет точность.
maxLevel: Больше уровней пирамиды — лучше для больших движений, но дольше.




Итог
Твой код — это классическая реализация Лукаса-Канаде для отслеживания движения ключевых точек. Он простой, но мощный для задач, где нужно понять, как движутся отдельные части сцены. Лукас-Канаде хорош для маленьких движений и заметных точек, но для сложных сцен (много объектов, быстрое движение, изменения света) могут понадобиться другие методы или доработки. Если хочешь, могу помочь настроить параметры, добавить фильтры или попробовать другой метод, например Farneback!

In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

# Параметры для выделения углов (feature detection)
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
# Параметры Лукаса-Канаде
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Получить первый кадр
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Маска для отрисовки траекторий
mask = np.zeros_like(old_frame)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Вычисление оптического потока
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    if p1 is not None and st is not None:
        good_new = p1[st==1]
        good_old = p0[st==1]

        # Отрисовка траекторий
        for i,(new,old) in enumerate(zip(good_new, good_old)):
            a,b = new.ravel()
            c,d = old.ravel()
            mask = cv2.line(mask, (int(a),int(b)), (int(c),int(d)), (0,255,0), 2)
            frame = cv2.circle(frame, (int(a),int(b)), 5, (0,0,255), -1)
        img = cv2.add(frame, mask)
    else:
        img = frame

    cv2.imshow('Optical Flow (Lucas-Kanade)', img)
    if cv2.waitKey(30) & 0xFF == 27:
        break

    old_gray = frame_gray.copy()
    p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

cap.release()
cv2.destroyAllWindows()

Что улучшено в этом варианте

Обработка ошибок на старте: Проверяем, удалось ли захватить первый кадр, и выходим с сообщением, если нет.
Переинициализация точек только при необходимости: Вместо поиска новых точек каждый кадр (что замедляет и сбрасывает траектории), мы добавляем новые только если осталось меньше 20 хороших (min_points). Это делает отслеживание более стабильным.
Фильтрация по ошибке: Используем err из calcOpticalFlowPyrLK — отбрасываем точки с ошибкой > 5 (можно настроить). Это уменьшает шум от неточных совпадений.
Случайные цвета для траекторий: Каждая точка получает уникальный цвет (из np.random), чтобы легче различать траектории разных объектов.
Периодическая очистка маски: Каждые 50 кадров (mask_clear_interval) маска обнуляется, чтобы старые траектории не загромождали экран вечно. Можно изменить интервал.
Обновление p0 и colors: Только хорошие точки переносятся в следующий кадр, и цвета сохраняются.
Обработка пустых p0: Если точек нет, ищем новые сразу, чтобы избежать ошибок.

Этот код должен работать плавнее, быть визуально информативнее и меньше "терять" точки. Если камера низкого качества, настрой порог ошибки или min_points. Тестируй на своей машине!

In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

# Параметры для выделения углов (feature detection)
feature_params = dict(maxCorners=100,
                      qualityLevel=0.3,
                      minDistance=7,
                      blockSize=7)

# Параметры Лукаса-Канаде
lk_params = dict(winSize=(15, 15),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Получить первый кадр
ret, old_frame = cap.read()
if not ret:
    print("Не удалось захватить первый кадр. Проверьте камеру.")
    cap.release()
    cv2.destroyAllWindows()
    exit()

old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

# Маска для отрисовки траекторий
mask = np.zeros_like(old_frame)

# Счётчик кадров для периодической очистки маски
frame_count = 0
mask_clear_interval = 50  # Очищать маску каждые 50 кадров, чтобы траектории не накапливались вечно

# Порог для переинициализации точек (если осталось меньше, ищем новые)
min_points = 20

# Массив случайных цветов для траекторий (по одной на точку)
if p0 is not None:
    colors = np.random.randint(0, 255, size=(len(p0), 3))

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if p0 is None or len(p0) == 0:
        # Если точек нет, ищем новые
        p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)
        if p0 is not None:
            colors = np.random.randint(0, 255, size=(len(p0), 3))
        continue

    # Вычисление оптического потока
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    if p1 is not None and st is not None and err is not None:
        # Фильтруем по статусу и ошибке (игнорируем точки с большой ошибкой)
        good_mask = (st.flatten() == 1) & (err.flatten() < 5)  # Порог ошибки можно настроить
        good_new = p1[good_mask]
        good_old = p0[good_mask]
        good_colors = colors[good_mask]  # Сохраняем цвета для хороших точек

        # Отрисовка траекторий с разными цветами
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            color = tuple(int(c) for c in good_colors[i])
            mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color, 2)
            frame = cv2.circle(frame, (int(a), int(b)), 5, color, -1)

        img = cv2.add(frame, mask)

        # Обновляем p0 и colors только хорошими точками
        p0 = good_new.reshape(-1, 1, 2)
        colors = good_colors
    else:
        img = frame

    # Периодическая очистка маски
    frame_count += 1
    if frame_count % mask_clear_interval == 0:
        mask = np.zeros_like(old_frame)

    # Переинициализация точек, если их осталось мало
    if len(p0) < min_points:
        new_points = cv2.goodFeaturesToTrack(frame_gray, mask=None, **feature_params)
        if new_points is not None:
            p0 = np.concatenate((p0, new_points), axis=0) if len(p0) > 0 else new_points
            new_colors = np.random.randint(0, 255, size=(len(new_points), 3))
            colors = np.concatenate((colors, new_colors), axis=0) if len(colors) > 0 else new_colors

    cv2.imshow('Optical Flow (Lucas-Kanade Improved)', img)
    if cv2.waitKey(30) & 0xFF == 27:
        break

    old_gray = frame_gray.copy()

cap.release()
cv2.destroyAllWindows()

3. Применить к видео любой из трекеров отслеживания, реализованных в 
библиотеках компьютерного зрения. 


Что такое трекинг объектов и зачем он нужен
Трекинг объектов (Object Tracking) — это процесс в компьютерном зрении, который позволяет следить за конкретным объектом (или областью) в последовательности кадров видео. В отличие от детекции движения (как в твоём первом коде с MOG2) или оптического потока (как во втором с Лукасом-Канаде), трекинг фокусируется на отслеживании определённого объекта, который пользователь выбирает. Это полезно в:

Видеонаблюдении: следить за человеком или машиной в толпе.
Робототехнике: робот отслеживает цель (например, мяч).
Анализе видео: отслеживать игроков в спортивных трансляциях или животных в исследованиях.
Интерфейсах: например, отслеживать лицо для фильтров в приложениях.
Твой код реализует трекинг с использованием нескольких алгоритмов OpenCV, позволяя пользователю выбрать объект с помощью ROI (Region of Interest, область интереса) и переключаться между разными трекерами (MIL, KCF, TLD, MOSSE, CSRT). Давай разберём теорию, термины и как это работает простыми словами.

Ключевые термины (терминология)
Трекинг объектов (Object Tracking): Процесс, при котором алгоритм запоминает выбранный объект (по его внешнему виду или особенностям) и следит за ним в новых кадрах, определяя его положение.
ROI (Region of Interest): Прямоугольная область на кадре, которую пользователь выбирает для отслеживания. В коде это initBB — координаты и размер прямоугольника (x, y, ширина, высота).
Трекер: Алгоритм, который отслеживает объект. OpenCV поддерживает несколько трекеров, таких как MIL, KCF, TLD, MOSSE, CSRT, каждый со своими плюсами и минусами.
Bounding Box (BB): Прямоугольник, который окружает отслеживаемый объект. В коде это box — координаты (x, y, w, h), где w — ширина, h — высота.
Tracker API в OpenCV: Интерфейс в OpenCV для трекинга, который включает методы init (инициализация трекера с начальным кадром и ROI) и update (обновление положения объекта в новом кадре).
Legacy Trackers: Старые версии трекеров в OpenCV, доступные через cv2.legacy. Некоторые алгоритмы (например, MIL, TLD) в новых версиях OpenCV перемещены в legacy, чтобы поддерживать обратную совместимость.
Кадр (Frame): Один снимок из видео, как в предыдущих кодах.
OpenCV: Библиотека компьютерного зрения, которая предоставляет готовые трекеры и функции для работы с видео.
Ключевые алгоритмы трекинга (в коде):
MIL (Multiple Instance Learning): Учитывает несколько "версий" объекта, чтобы быть устойчивым к изменениям внешнего вида (например, поворот объекта). Хорошо работает с частичными перекрытиями, но не всегда точен.
KCF (Kernelized Correlation Filters): Использует корреляционные фильтры для быстрого и точного трекинга. Хорошо справляется с изменениями масштаба и поворотами, но может терять объект при сильных перекрытиях.
TLD (Tracking-Learning-Detection): Комбинирует трекинг с обучением и детекцией, чтобы находить объект, даже если он временно пропадает. Устойчив к исчезновениям, но медленнее.
MOSSE (Minimum Output Sum of Squared Error): Очень быстрый, но менее точный. Подходит для простых задач, где объект не сильно меняется.
CSRT (Channel and Spatial Reliability Tracking): Более точный, использует информацию о цвете и пространственную надёжность. Медленнее, но лучше справляется со сложными сценами.
Как работает код (пошагово и просто)
Твой код позволяет пользователю выбрать объект для отслеживания с помощью мыши (ROI) и переключать трекеры (1-5 на клавиатуре). Он использует API трекинга OpenCV. Вот как это работает:

Инициализация:
Код устанавливает зависимости (opencv-contrib-python), так как некоторые трекеры (например, CSRT) доступны только в contrib.
Определяет словарь TRACKERS, связывающий клавиши (1-5) с названиями трекеров (MIL, KCF, TLD, MOSSE, CSRT).
Функция create_tracker создаёт трекер по имени, сначала проверяя cv2.legacy, потом cv2, чтобы учесть разные версии OpenCV.
По умолчанию выбирается трекер MIL (tracker_type = 'MIL').
Открывается веб-камера (cv2.VideoCapture(0)), и initBB (начальный bounding box) пока пустой.
Основной цикл:
Читает кадр (ret, frame = cap.read()).
Показывает текущий трекер на экране (cv2.putText с текстом "Tracker: ...").
Если ROI уже выбран (initBB не None):
Вызывает tracker.update(frame), чтобы найти новое положение объекта.
Если трекинг успешен (success=True), рисует зелёный прямоугольник вокруг объекта (cv2.rectangle).
Если трекинг провалился (success=False), показывает сообщение "Tracking failure".
Показывает кадр с результатом (cv2.imshow).
Обработка ввода пользователя:
Клавиша 's': Запускает cv2.selectROI, чтобы пользователь мог выделить объект мышью. Если ROI выбран (не нулевой), инициализирует трекер (tracker.init(frame, initBB)).
Клавиши 1-5: Меняют тип трекера (например, 1 — MIL, 2 — KCF). Если ROI уже есть, новый трекер инициализируется с тем же initBB.
Esc (27): Завершает программу.
Цикл продолжается, пока не нажат Esc или не закончено видео.
Завершение: Закрывает камеру и окна.
Что делает трекер внутри:

При инициализации (tracker.init) трекер "запоминает" внешний вид объекта в ROI (цвета, текстуры, особенности).
На каждом кадре (tracker.update) он ищет область, наиболее похожую на начальную, и возвращает новый bounding box.
Каждый трекер использует свой алгоритм (MIL, KCF и т.д.), чтобы справляться с изменениями внешнего вида, поворотами или перекрытиями.
Теория: как работают трекеры
Каждый трекер решает задачу: "где этот объект в новом кадре?". Они отличаются подходами:

MIL: Считает, что объект может немного меняться (например, из-за света). Учитывает несколько "гипотез" о том, как объект выглядит. Хорошо для нестабильных условий, но может "плавать".
KCF: Использует корреляцию (сравнение шаблона) и машинное обучение для быстрого поиска объекта. Эффективен, но теряет объект, если он сильно закрыт.
TLD: Комбинирует трекинг с детекцией. Если объект пропадает (например, вышел из кадра), пытается найти его снова. Подходит для длительного трекинга.
MOSSE: Максимально быстрый, использует корреляцию с минимальной ошибкой. Хорош для простых задач, но неустойчив к изменениям.
CSRT: Учитывает цвета и пространственную информацию, более точный, но медленный. Лучше для сложных сцен с перекрытиями.
Общий принцип:

Трекеры запоминают "шаблон" объекта (по пикселям или особенностям, как края).
На каждом кадре ищут, где этот шаблон лучше всего совпадает.
Возвращают новый bounding box или флаг, что объект потерян.
Ограничения:

Трекеры могут терять объект при сильных изменениях (освещение, поворот, перекрытие).
Они не "понимают", что отслеживают (например, человек это или машина), в отличие от нейросетей вроде YOLO.
Производительность зависит от сложности сцены и мощности компьютера.
Что ещё есть: альтернативы и улучшения
Другие трекеры в OpenCV:
Boosting: Похож на MIL, но старше и медленнее. Доступен в cv2.legacy.
MedianFlow: Быстрый, но требует, чтобы объект двигался предсказуемо. Хорош для плавных движений.
GOTURN: Нейросетевой трекер в OpenCV (требует contrib и веса модели). Точнее, но сложнее в настройке.
Альтернативы вне OpenCV:
DeepSORT: Современный трекер на основе нейросетей. Используется с YOLO для точного отслеживания (например, людей в толпе).
Siamese Networks: Нейросетевые трекеры, которые сравнивают шаблон объекта с новым кадром. Очень точные, но требуют GPU.
Улучшения для кода:
Сохранение траектории: Можно рисовать путь объекта (как в коде с Лукасом-Канаде) с помощью маски.
Фильтрация по цвету: Добавить фильтр HSV, чтобы трекер фокусировался на объекте определённого цвета.
Перезапуск при потере: Если трекинг провалился, автоматически искать объект снова (как в TLD).
Мультитрекинг: Использовать cv2.legacy.MultiTracker для отслеживания нескольких объектов.
Улучшение ROI: Добавить проверку, чтобы ROI не был слишком маленьким (например, if w > 10 and h > 10).
Итог
Твой код — это удобная демонстрация трекинга объектов с переключением алгоритмов. Он позволяет пользователю выбирать объект и тестировать разные трекеры, что полезно для сравнения их работы. MIL — хороший старт, но CSRT обычно точнее, а MOSSE — быстрее. Для реальных задач можно добавить фильтрацию, мультитрекинг или комбинировать с детекцией (например, YOLO). Если хочешь, могу предложить конкретный улучшенный код с одной из этих идей или объяснить какой-то трекер подробнее!


## **MIL (Multiple Instance Learning)**

**Как работает:**
- Вместо одного положительного примера использует "пакеты" примеров
- Для каждого кадра собирает несколько положительных образцов вокруг текущей позиции цели и отрицательные - дальше от цели
- Обущает классификатор отличать цель от фона
- **Ключевая идея**: даже если некоторые примеры в пакете неправильные, классификатор учится на целом наборе

**Сильные стороны:**
- Устойчив к неточностям разметки
- Хорошо справляется с частичными перекрытиями
- Менее чувствителен к дрожанию

## **KCF (Kernelized Correlation Filters)**

**Как работает:**
1. **Корреляционные фильтры**: обучает фильтр, который дает максимальный отклик на цели и минимальный на фоне
2. **Циклические сдвиги**: создает тренировочные примеры циклическим сдвигом исходного изображения (эффективно по памяти)
3. **Ядерный трюк**: использует ядра (Kernel trick) для работы в высокоразмерном пространстве без явного вычисления признаков
4. **Быстрое преобразование**: вычисления в частотной области через FFT

**Сильные стороны:**
- Невероятно быстрый (сотни FPS)
- Точнее чем MOSSE
- Эффективное использование памяти

## **TLD (Tracking-Learning-Detection)**

**Три взаимосвязанных компонента:**

1. **Tracker** (слежение):
   - Оптический поток между кадрами
   - Предсказывает положение в следующем кадре

2. **Detector** (детектирование):
   - Каскадный классификатор по типу Viola-Jones
   - Ищет цель по всему кадру если трекер потерял

3. **Learner** (обучение):
   - Анализирует ошибки трекера и детектора
   - Обновляет модель цели онлайн
   - Добавляет новые положительные/отрицательные примеры

**Сильные стороны:**
- Самовосстановление после потери цели
- Адаптация к изменению внешнего вида
- Очень надежный в сложных сценариях

## **MOSSE (Minimum Output Sum of Squared Error)**

**Как работает:**
1. **Инициализация**: пользователь выделяет цель в первом кадре
2. **Обучение фильтра**: находит фильтр, который минимизирует ошибку между желаемым откликом и фактическим
3. **Свертка**: применяет обученный фильтр к новому кадру
4. **Пик отклика**: позиция с максимальным откликом - новое положение цели
5. **Адаптация**: постепенно обновляет фильтр для учета изменений

**Сильные стороны:**
- Экстремально быстрый (до 600 FPS)
- Устойчив к изменениям освещения
- Малое потребление памяти

## **CSRT (Channel and Spatial Reliability Tracking)**

**Как работает:**
1. **Цветовые пространства**: использует не только HOG, но и цветовые характеристики
2. **Пространственная надежность**: вычисляет надежность разных регионов цели
3. **Надежность каналов**: определяет какие цветовые каналы наиболее информативны
4. **Адаптивное окно**: автоматически подстраивает размер окна отслеживания

**Сильные стороны:**
- Высокая точность позиционирования
- Автоматическое определение масштаба
- Устойчив к деформациям и вращению

## **Сравнение производительности:**
- **Скорость**: MOSSE > KCF > CSRT > MIL > TLD
- **Точность**: CSRT > TLD > KCF > MIL > MOSSE  
- **Надежность**: TLD > CSRT > MIL > KCF > MOSSE

Выбор зависит от задачи: скорость (MOSSE/KCF) vs точность (CSRT) vs надежность (TLD).

In [7]:
#%pip uninstall opencv-python opencv-contrib-python -y
#%pip install opencv-contrib-python --no-cache-dir

import cv2

TRACKERS = {
    '1': 'MIL',
    '2': 'KCF',
    '3': 'TLD',
    '4': 'MOSSE',
    '5': 'CSRT',
}

def create_tracker(tracker_type):
    # Try legacy first
    if hasattr(cv2.legacy, f'Tracker{tracker_type}_create'):
        creator = getattr(cv2.legacy, f'Tracker{tracker_type}_create')
        return creator()
    # Fallback to direct cv2
    elif hasattr(cv2, f'Tracker{tracker_type}_create'):
        creator = getattr(cv2, f'Tracker{tracker_type}_create')
        return creator()
    raise Exception(f"{tracker_type} tracker is not available in your OpenCV installation.")

# Default tracker
tracker_type = 'MIL'
tracker = create_tracker(tracker_type)

cap = cv2.VideoCapture(0)
initBB = None

while True:
    ret, frame = cap.read()
    if not ret:
        break

    cv2.putText(frame, f"Tracker: {tracker_type} (Press 1-5 to change)", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

    if initBB is not None:
        success, box = tracker.update(frame)
        if success:
            (x, y, w, h) = [int(v) for v in box]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        else:
            cv2.putText(frame, "Tracking failure", (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    cv2.imshow("Tracking", frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord("s"):
        initBB = cv2.selectROI("Tracking", frame, fromCenter=False, showCrosshair=True)
        if initBB:  # Only init if ROI selected (non-zero)
            tracker = create_tracker(tracker_type)
            tracker.init(frame, initBB)
    elif chr(key) in TRACKERS:  # Check if key is '1' to '7'
        new_tracker_type = TRACKERS[chr(key)]
        if new_tracker_type != tracker_type:
            tracker_type = new_tracker_type
            if initBB is not None:
                tracker = create_tracker(tracker_type)
                tracker.init(frame, initBB)
    elif key == 27:  # ESC to quit
        break

cap.release()
cv2.destroyAllWindows()

# Контрольные вопросы

### 1. Принцип действия Background Subtraction Methods
Это методы, которые строят модель "фона" (статической сцены), а затем на каждом кадре вычитают фон, выделяя таким образом "новые" (движущиеся) объекты. Пример: MOG2 моделирует фон с помощью гауссовых смесей, обновляя модель по мере поступления новых кадров.

### 2. Принцип действия метода Лукаса-Канаде
Это метод оценки оптического потока (движения яркостных пятен между кадрами). Считается, что соседние пиксели движутся одинаково (локальное постоянство), и используется аппроксимация движения на небольшом окне. Обычно применяется к выделенным ключевым точкам.

### 3. В чем различие методов детектирования движения от методов отслеживания (трекеров)?
Детектирование движения — это поиск областей, где что-то изменилось (например, появление нового объекта), а трекинг — это отслеживание уже найденного объекта на следующих кадрах, даже если его форма/размер меняются.

### 4. Каков принцип работы трекеров движущихся объектов?
Трекер получает стартовое положение объекта и далее на каждом кадре ищет его новое положение, используя алгоритмы сопоставления (по цвету, текстуре, шаблону и др.), что позволяет отслеживать даже частично перекрытые или изменившиеся объекты.